## Brain Captioning

Using the GIT model we want to predict image captions from brain activity

In [1]:
from transformers import AutoProcessor
from modeling_git import GitForCausalLM, GitModel, GitForCausalLMClipEmb
import requests
from PIL import Image
import numpy as np
import os
import glob
from os.path import join as opj
import h5py  
import matplotlib.pyplot as plt
import pandas as pd
import nibabel as nib
from scipy.io import loadmat
import torch
from torch.utils.data import Dataset, Subset, DataLoader
import json
from torchsummary import summary
import torchvision
import tqdm
from sklearn.linear_model import Ridge
import pickle
import wandb
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

from yellowbrick.cluster import KElbowVisualizer

## vdvae
import pickle


from hps import Hyperparams
from vae import VAE

from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.manifold import TSNE
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud
import numpy as np
from transformers import AutoImageProcessor, UperNetForSemanticSegmentation
from PIL import Image
import numpy as np
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from torchmetrics import BLEUScore
import evaluate
from sentence_transformers import SentenceTransformer, util
from transformers import AutoProcessor, CLIPModel, AutoTokenizer
from PIL import ImageFilter
# from controlnet import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: matteoferrante. Use `wandb login --relogin` to force relogin


True

In [3]:
base_path="/home/matteo/brain-diffuser/data"
timeseries_path=opj(base_path,"nsddata_timeseries")
betas_path=opj(base_path,"nsddata_betas")

stimuli_path=opj(base_path,"nsddata_stimuli","stimuli","nsd")
stim_file_path=opj(stimuli_path,"nsd_stimuli.hdf5")
sub="subj05"
mod="func1pt8mm"
subj_data_path=opj(timeseries_path,"ppdata",sub,mod,"timeseries")
subj_betas_path=opj(betas_path,"ppdata",sub,mod,"betas_assumehrf")

subj_betas_roi_extracted_path=opj(base_path,"processed_roi",sub,mod)

stim_order_path=opj(base_path,"nsddata","experiments","nsd","nsd_expdesign.mat")
stim_info_path=opj(base_path,"nsddata","experiments","nsd","nsd_stim_info_merged.csv")
stim_captions_train_path=opj(base_path,"nsddata_stimuli","stimuli","nsd","annotations",f"captions_train2017.json")
stim_captions_val_path=opj(base_path,"nsddata_stimuli","stimuli","nsd","annotations",f"captions_val2017.json")

processed_data=opj(base_path,"processed_data",sub)
sub_idx=int(sub.split("0")[-1])

fmri_train_data=opj(processed_data,f"nsd_train_fmriavg_nsdgeneral_sub{sub_idx}.npy")
imgs_train_data=opj(processed_data,f"nsd_train_stim_sub{sub_idx}.npy")
captions_train_data=opj(processed_data, f"nsd_train_cap_sub{sub_idx}.npy")
       
fmri_test_data=opj(processed_data,f"nsd_test_fmriavg_nsdgeneral_sub{sub_idx}.npy")
imgs_test_data=opj(processed_data,f"nsd_test_stim_sub{sub_idx}.npy")
captions_test_data=opj(processed_data, f"nsd_test_cap_sub{sub_idx}.npy")
                       

In [4]:
compute_dataset=True
train=True
adjust=True

In [5]:
class NSDDataset(Dataset):
    

    
    def __init__(self, fmri_data,imgs_data,caption_data,transforms=None):
        self.fmri_data=np.load(fmri_data)
        self.imgs_data=np.load(imgs_data).astype(np.uint8)
        self.caption_data=np.load(caption_data,allow_pickle=True)
        self.transforms=transforms
        
    def __len__(self):
        return  len(self.fmri_data)
    
    def __getitem__(self,idx):
        fmri=torch.tensor(self.fmri_data[idx])
        img=Image.fromarray(self.imgs_data[idx])
        
        if self.transforms:
            img=self.transforms(img)
        
        caption=self.caption_data[idx][0] #cambiare se ne voglio altre
        
        return fmri,img,caption

In [6]:
tr=torchvision.transforms.ToTensor()
train_dataset=NSDDataset(fmri_train_data,imgs_train_data,captions_train_data,transforms=tr)
test_dataset=NSDDataset(fmri_test_data,imgs_test_data,captions_test_data,transforms=tr)

BS=128

train_dataloader=DataLoader(train_dataset,BS,shuffle=True)
test_dataloader=DataLoader(test_dataset,BS,shuffle=False)


In [7]:
to_pil=torchvision.transforms.ToPILImage()

In [8]:
device="cuda:0"
processor = AutoProcessor.from_pretrained("microsoft/git-base-coco")
model = GitForCausalLMClipEmb.from_pretrained("microsoft/git-base-coco")

model.to(device)
url = "prova.png"
image = Image.open(url)

pixel_values = processor(images=image, return_tensors="pt").pixel_values
vision_encoder=model.git.image_encoder


In [9]:
model_base = GitForCausalLM.from_pretrained("microsoft/git-base-coco")

model_base.to(device)



GitForCausalLM(
  (git): GitModel(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
          (position_embedding): Embedding(197, 768)
        )
        (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-11): 12 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): Linear(in_features=768, out_features=768, bias=True)
             

In [10]:
if compute_dataset:
    train_fmri=[]
    train_imgs=[]
    train_captions=[]
    train_clip_img_embeds=[]


    for x,y,c in tqdm.tqdm(train_dataloader):

        #save fMRI data
        train_fmri.append(x)

        #save img data
        train_imgs.append(y)

        train_captions+=list(c)

        #encode images in autoencoder and save z representation
        with torch.no_grad():
            
            #encode images in CLIP
            
            pixel_values= processor(images=y, return_tensors="pt").pixel_values.to(device)
            
            image_features=vision_encoder(pixel_values).last_hidden_state.cpu()
            train_clip_img_embeds.append(image_features)

           
    train_clip_img_embeds = torch.cat(train_clip_img_embeds,axis=0)

    train_fmri = torch.cat(train_fmri,axis=0)
    train_imgs = torch.cat(train_imgs,axis=0)


100%|████████████████████████████████████████████████████████████| 70/70 [05:58<00:00,  5.12s/it]


In [11]:
if compute_dataset:
    test_fmri=[]
    test_imgs=[]
    test_captions=[]
    test_clip_img_embeds=[]


    for x,y,c in tqdm.tqdm(test_dataloader):

        #save fMRI data
        test_fmri.append(x)

        #save img data
        test_imgs.append(y)

        test_captions+=list(c)

        #encode images in autoencoder and save z representation
        with torch.no_grad():
            
            #encode images in CLIP
            
            pixel_values= processor(images=y, return_tensors="pt").pixel_values.to(device)
            
            image_features=vision_encoder(pixel_values).last_hidden_state.cpu()
            test_clip_img_embeds.append(image_features)

           
    test_clip_img_embeds = torch.cat(test_clip_img_embeds,axis=0)

    test_fmri = torch.cat(test_fmri,axis=0)
    test_imgs = torch.cat(test_imgs,axis=0)

100%|██████████████████████████████████████████████████████████████| 8/8 [00:12<00:00,  1.54s/it]


In [12]:
if compute_dataset:
    os.makedirs(f"models/{sub}",exist_ok=True)
    
    ## train
    torch.save(train_fmri,f"models/{sub}/train_fmri.pt")
    torch.save(train_clip_img_embeds,f"models/{sub}/train_clip_img_embeds.pt")
    torch.save(train_imgs,f"models/{sub}/train_imgs.pt")
        
    with open(f"models/{sub}/train_captions.sav","wb") as f:
        pickle.dump(train_captions,f)
        
    print("saved training stuff")
    
    ## test
    torch.save(test_fmri,f"models/{sub}/test_fmri.pt")
    torch.save(test_clip_img_embeds,f"models/{sub}/test_clip_img_embeds.pt")
    torch.save(test_imgs,f"models/{sub}/test_imgs.pt")
        
    with open(f"models/{sub}/test_captions.sav","wb") as f:
        pickle.dump(test_captions,f)
    
    
    print("saved testing stuff")
    
else:
    if sub=="subj01_good2":
        sub="subj01"
    ## train
    train_fmri=torch.load(f"models/{sub}/train_fmri.pt")
    train_clip_img_embeds= torch.load(f"models/{sub}/train_clip_img_embeds.pt")
    train_imgs=torch.load(f"models/{sub}/train_imgs.pt")
        
    with open(f"models/{sub}/train_captions.sav","rb") as f:
        train_captions=pickle.load(f)

    ## test
    test_fmri=torch.load(f"models/{sub}/test_fmri.pt")
    test_clip_img_embeds= torch.load(f"models/{sub}/test_clip_img_embeds.pt")
    test_imgs=torch.load(f"models/{sub}/test_imgs.pt")
    
    with open(f"models/{sub}/test_captions.sav","rb") as f:
        test_captions=pickle.load(f)
    

saved training stuff
saved testing stuff


In [13]:
train_fmri_mean=torch.mean(train_fmri,axis=0)
train_fmri_std=torch.std(train_fmri,axis=0)

train_fmri_norm=(train_fmri-train_fmri_mean)/train_fmri_std
# val_fmri_norm=(val_fmri-train_fmri_mean)/train_fmri_std
test_fmri_norm=(test_fmri-train_fmri_mean)/train_fmri_std


In [14]:
max_len_img=197

if train:
    brain_to_img_emb=[]

    for i in tqdm.tqdm(range(max_len_img)):
        m=Ridge(alpha=6e4)
        m.fit(train_fmri_norm.numpy(),train_clip_img_embeds[:,i,:].numpy())
        brain_to_img_emb.append(m)

100%|██████████████████████████████████████████████████████████| 197/197 [57:36<00:00, 17.54s/it]


In [15]:
if train:
    os.makedirs(f"models/{sub}/decoding",exist_ok=True)
    for i in range(max_len_img):
        filename = f'brain_to_img_emb_ridge_{i}.sav'
        with open(opj(f"models/{sub}/decoding",filename), 'wb') as f:
            pickle.dump(brain_to_img_emb[i], f) 
else:
    brain_to_img_emb=[]
    for i in range(max_len_img):
        filename = f'brain_to_img_emb_ridge_{i}.sav'
        with open(opj(f"models/{sub}/decoding",filename), 'rb') as f:
            p=pickle.load(f)
            brain_to_img_emb.append(p)

In [16]:
if train:
    img_emb_train=[]
    for i in tqdm.tqdm(range(max_len_img)):
        emb=torch.tensor(brain_to_img_emb[i].predict(train_fmri_norm.numpy()))


        img_emb_train.append(emb)
    img_emb_train=torch.stack(img_emb_train,1)


100%|██████████████████████████████████████████████████████████| 197/197 [01:20<00:00,  2.46it/s]


In [17]:
if train:
    
    train_clip_img_embeds_mean=train_clip_img_embeds.mean(0)
    train_clip_img_embeds_std=train_clip_img_embeds.std(0)
    
    pred_clip_img_embeds_mean=img_emb_train.mean(0)
    pred_clip_img_embeds_std=img_emb_train.std(0)
    
    torch.save(train_clip_img_embeds_mean, opj(f"models/{sub}","train_clip_img_embeds_mean.pt"))
    torch.save(train_clip_img_embeds_std, opj(f"models/{sub}","train_clip_img_embeds_std.pt"))
    torch.save(pred_clip_img_embeds_mean, opj(f"models/{sub}","pred_clip_img_embeds_mean.pt"))
    torch.save(pred_clip_img_embeds_std, opj(f"models/{sub}","pred_clip_img_embeds_std.pt"))
    
else:
    train_clip_img_embeds_mean=torch.load(opj(f"models/{sub}","train_clip_img_embeds_mean.pt"))
    train_clip_img_embeds_std=torch.load(opj(f"models/{sub}","train_clip_img_embeds_std.pt"))
   

    pred_clip_img_embeds_mean=torch.load(opj(f"models/{sub}","pred_clip_img_embeds_mean.pt"))
    pred_clip_img_embeds_std=torch.load(opj(f"models/{sub}","pred_clip_img_embeds_std.pt"))
    
    

In [18]:

img_emb_test=[]
for i in tqdm.tqdm(range(max_len_img)):
    emb=torch.tensor(brain_to_img_emb[i].predict(test_fmri_norm.numpy()))


    img_emb_test.append(emb)
img_emb_test=torch.stack(img_emb_test,1)


100%|██████████████████████████████████████████████████████████| 197/197 [00:36<00:00,  5.35it/s]


In [19]:
if adjust:
    img_emb_test_adj=(img_emb_test-img_emb_test.mean(0))/(img_emb_test.std(0))
    img_emb_test_adj=train_clip_img_embeds_std*img_emb_test_adj+train_clip_img_embeds_mean

In [20]:
captions_from_images=[]
captions_from_brain=[]

with torch.no_grad():
    for i in tqdm.tqdm(range(0,len(test_imgs),BS)):

        #get reference images
        imgs=[to_pil(j) for j in test_imgs[i:i+BS]]

        #compute captions from images
        pixel_values = processor(images=imgs, return_tensors="pt").pixel_values
        generated_ids = model_base.generate(pixel_values=pixel_values.to(device), max_length=25)
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)

        captions_from_images+=generated_caption

        #compute captions from brain
        generated_ids = model.generate(pixel_values=img_emb_test_adj[i:i+BS].to(device).float(), max_length=25)
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
        captions_from_brain+=generated_caption

100%|██████████████████████████████████████████████████████████████| 8/8 [02:18<00:00, 17.36s/it]


## Compute text metrics
* BLEU
* METEOR
* Sentence
* CLIP

In [21]:
bleu=evaluate.load("bleu")
meteor = evaluate.load('meteor')
bleu_img_ref=bleu.compute(predictions=captions_from_images,references=test_captions,max_order=1)
bleu_brain_ref=bleu.compute(predictions=captions_from_brain,references=test_captions,max_order=1)
bleu_brain_img=bleu.compute(predictions=captions_from_brain,references=captions_from_images,max_order=1)

bleu_img_ref_4=bleu.compute(predictions=captions_from_images,references=test_captions,max_order=4)
bleu_brain_ref_4=bleu.compute(predictions=captions_from_brain,references=test_captions,max_order=4)
bleu_brain_img_4=bleu.compute(predictions=captions_from_brain,references=captions_from_images,max_order=4)



relative_brain_image_bleu=bleu_brain_img["bleu"]/bleu_img_ref["bleu"]
relative_brain_image_bleu_4=bleu_brain_img_4["bleu"]/bleu_img_ref_4["bleu"]


print(f"[GROUND] BLEU@1 GIT from images vs captions: {bleu_img_ref['bleu']}")
print(f"[ABSOLUTE] BLEU@1 GIT from brain vs images: {bleu_brain_img['bleu']}")
print(f"[RELATIVE] BLEU@1  {relative_brain_image_bleu}")

print()
print(f"[GROUND] BLEU@4 GIT from images vs captions: {bleu_img_ref_4['bleu']}")
print(f"[ABSOLUTE] BLEU@4 GIT from brain vs images: {bleu_brain_img_4['bleu']}")
print(f"[RELATIVE] BLEU@4  {relative_brain_image_bleu_4}")


[nltk_data] Downloading package wordnet to /home/matteo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/matteo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/matteo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[GROUND] BLEU@1 GIT from images vs captions: 0.43787420529004933
[ABSOLUTE] BLEU@1 GIT from brain vs images: 0.4335066205372695
[RELATIVE] BLEU@1  0.9900254806060413

[GROUND] BLEU@4 GIT from images vs captions: 0.13830765150649799
[ABSOLUTE] BLEU@4 GIT from brain vs images: 0.11177697624503605
[RELATIVE] BLEU@4  0.8081763736678338


In [22]:
meteor_img_ref=meteor.compute(predictions=captions_from_images,references=test_captions)
meteor_brain_ref=meteor.compute(predictions=captions_from_brain,references=test_captions)
meteor_brain_img=meteor.compute(predictions=captions_from_brain,references=captions_from_images)


relative_brain_image_meteor=meteor_brain_img["meteor"]/meteor_img_ref["meteor"]

print(f"[GROUND] METEOR GIT from images vs captions: {meteor_img_ref['meteor']}")
print(f"[ABSOLUTE] METEOR GIT from brain vs images: {meteor_brain_img['meteor']}")
print(f"[RELATIVE] METEOR  {relative_brain_image_meteor}")



[GROUND] METEOR GIT from images vs captions: 0.4038153815765378
[ABSOLUTE] METEOR GIT from brain vs images: 0.3038103202360226
[RELATIVE] METEOR  0.7523495490684756


In [23]:
sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [24]:
#Compute embedding for both lists
with torch.no_grad():
    embedding_brain= sentence_model.encode(captions_from_brain, convert_to_tensor=True)
    embedding_captions = sentence_model.encode(test_captions, convert_to_tensor=True)
    embedding_images = sentence_model.encode(captions_from_images, convert_to_tensor=True)

    ss_sim_brain_img=util.pytorch_cos_sim(embedding_brain, embedding_images).cpu()
    ss_sim_brain_cap=util.pytorch_cos_sim(embedding_brain, embedding_captions).cpu()
    ss_sim_img_cap=util.pytorch_cos_sim(embedding_images, embedding_captions).cpu()

    relative_brain_image_ss=ss_sim_brain_img.diag().mean()/ss_sim_img_cap.diag().mean()

In [25]:
print(f"[GROUND] Sentence Transformer Similarity GIT from images vs captions: {ss_sim_img_cap.diag().mean()}")
print(f"[ABSOLUTE] Sentence Transformer Similarity GIT from brain vs images: {ss_sim_brain_img.diag().mean()}")
print(f"[RELATIVE] Sentence Transformer Similarity   {relative_brain_image_ss.mean()}")


[GROUND] Sentence Transformer Similarity GIT from images vs captions: 0.7029159069061279
[ABSOLUTE] Sentence Transformer Similarity GIT from brain vs images: 0.443602979183197
[RELATIVE] Sentence Transformer Similarity   0.6310896873474121


In [26]:
model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor_clip = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer =  AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

with torch.no_grad():
    input_ids=tokenizer(captions_from_brain,return_tensors="pt",padding=True)
    embedding_brain= model_clip.get_text_features(**input_ids)

    input_ids=tokenizer(test_captions,return_tensors="pt",padding=True)
    embedding_captions= model_clip.get_text_features(**input_ids)

    input_ids=tokenizer(captions_from_images,return_tensors="pt",padding=True)
    embedding_images= model_clip.get_text_features(**input_ids)

clip_sim_brain_img=util.pytorch_cos_sim(embedding_brain, embedding_images).cpu()
clip_sim_brain_cap=util.pytorch_cos_sim(embedding_brain, embedding_captions).cpu()
clip_sim_img_cap=util.pytorch_cos_sim(embedding_images, embedding_captions).cpu()

relative_brain_image_clip=clip_sim_brain_img.diag().mean()/clip_sim_img_cap.diag().mean()

print(f"[GROUND] CLIP Similarity GIT from images vs captions: {clip_sim_img_cap.diag().mean()}")
print(f"[ABSOLUTE] CLIP Similarity GIT from brain vs images: {clip_sim_brain_img.diag().mean()}")
print(f"[RELATIVE] CLIP Similarity   {relative_brain_image_clip.mean()}")

[GROUND] CLIP Similarity GIT from images vs captions: 0.8314802050590515
[ABSOLUTE] CLIP Similarity GIT from brain vs images: 0.7022090554237366
[RELATIVE] CLIP Similarity   0.8445289134979248


In [27]:
out_data={"ROI":"nsdgeneral",
          "Bleu_1_img_ref": bleu_img_ref['bleu'],
          "Bleu_1_brain_img": bleu_img_ref['bleu'],
          "Bleu_1_relative": bleu_img_ref['bleu'],
          "Bleu_4_img_ref": bleu_img_ref_4['bleu'],
          "Bleu_4_brain_img": bleu_img_ref_4['bleu'],
          "Bleu_4_relative": bleu_img_ref_4['bleu'],
          "Meteor_img_ref":meteor_img_ref['meteor'],
          "Meteor_brain_img":meteor_brain_img['meteor'],
          "Meteor_relative":relative_brain_image_meteor,
          "Sentence_img_ref":ss_sim_img_cap.diag().mean().item(),
          "Sentence_brain_img":ss_sim_brain_img.diag().mean().item(),
          "Sentence_relative":relative_brain_image_ss.mean().item(),
          "CLIP_img_ref":clip_sim_img_cap.diag().mean().item(),
          "CLIP_brain_img":clip_sim_brain_img.diag().mean().item(),
          "CLIP_relative":relative_brain_image_clip.mean().item()}

df=pd.DataFrame.from_dict([out_data])
df.to_csv(f"nsdgeneral_{sub}.csv")


### Save all to wandb

In [28]:
config={"sub":sub, "mod": mod, "model": "Ridge"}
wandb.init(project="BrainCaptioning", config=config)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
wandb.log(out_data)

## Perform Image Decoding

* Define styles
* Import StableDiffusion2
* Import StableDiffusion + ControlNet
* Load VDVAE for image reconstruction
* Load VDVAE for depth image reconstruction

In [30]:
styles=["illustration", "drawing", "cartoon", "hyperrealistic","fantasy", "surrealist"]

In [31]:
def get_latents(data,brain_to_latent,shapes,adjust=None):
    latents={}
    bs=data.shape[0]
    for k,v in brain_to_latent.items():
        s=shapes[k]
        z=torch.tensor(v.predict(data)).reshape(-1,*s)
        
    
        if adjust is not None and bs>1:
            #compute actual mean and std
            z_mean=z.mean(0)  
            z_std=z.std(0)
            #standardize 
            z = (z - z_mean)/(1e-9+z_std)
            
            #replace with latent mean and std
            z = z*adjust[k]["std"]+adjust[k]["mean"]
        
        latents[k]=z
        
    return latents

In [32]:
def decode_with_partial_sampling(decoder,latents,keep=31):
    xs = {a.shape[2]: a for a in decoder.bias_xs}
    
    out=vae.decoder.forward_manual_latents(keep,latents.values(),t=None)
    
    xs=decoder.out_net.sample(out)
    xs=torch.tensor(xs).permute(0,3,1,2)/255
    return xs

In [33]:
upsample=torchvision.transforms.Resize(512,interpolation=torchvision.transforms.InterpolationMode.BILINEAR)
to_pil=torchvision.transforms.ToPILImage()

In [34]:
if sub=="subj01":
    sub="subj01_good2"
with open(f"/home/matteo/explore_NSD/models/{sub}/train_z.sav","rb") as f:
    train_z=pickle.load(f)
    
with open(f"/home/matteo/explore_NSD/models/{sub}/test_z.sav","rb") as f:
    test_z=pickle.load(f)
shapes={k: (v.shape[1],v.shape[2],v.shape[3]) for k,v in train_z.items()}


In [35]:
keep=31
if sub=="subj01_good2":
    sub="subj01"
if not train:
    keys=np.arange(keep)
    # filename='brain_to_latent_ridge.sav'
    brain_to_depth = {}
#     pickle.load(open(opj(f"models/{sub}/decoding",filename), 'rb'))
    for k in keys:
        filename = f'brain_to_depth_vdvae_latent_ridge_{k}.sav'
        p=pickle.load(open(opj(f"/home/matteo/explore_NSD/models/{sub}/depth",filename), 'rb'))
        brain_to_depth[k]=p


In [36]:
keep=31
keys=np.arange(keep)
brain_to_latent = {}


for k in keys:
    filename = f'brain_to_vdvae_latent_ridge_{k}.sav'
    p=pickle.load(open(opj(f"/home/matteo/explore_NSD/models/{sub}/decoding",filename), 'rb'))
    brain_to_latent[k]=p

    
latent_adjust_values={}
for i in range(keep):
    latent_adjust_values[i]={"mean":train_z[i].mean(0), "std": train_z[i].std(0)}

In [37]:
filename = f'latent_depth_adjust_values.sav'
latent_depth_adjust_values={}

if sub=="subj01_good2":
    sub="subj01"

with open(opj(f"/home/matteo/explore_NSD/models/{sub}",filename), 'rb') as f:
    latent_depth_adjust_values=pickle.load( f)
    
    # explore_NSD/models/subj01/latent_depth_adjust_values.sav

In [38]:
# fix nan

for k in train_z.keys():
    train_z[k]=torch.nan_to_num(train_z[k])
    # val_z[k]=torch.nan_to_num(val_z[k])
    test_z[k]=torch.nan_to_num(test_z[k])
    

In [39]:
with open('/home/matteo/models/vdvae/H.sav', 'rb') as fp:
    d = pickle.load(fp)
    
H=Hyperparams()
for k,v in d.items():
    H[k]=v
    
vae=VAE(H)

state_dict = torch.load("/home/matteo/models/vdvae/vae2.pt")
new_state_dict = {}
l = len('module.')
for k in state_dict:
    if k.startswith('module.'):
        new_state_dict[k[l:]] = state_dict[k]
    else:
        new_state_dict[k] = state_dict[k]
state_dict = new_state_dict
vae.load_state_dict(state_dict)
vae=vae.to(device)

In [40]:
checkpoint_shuffle = "lllyasviel/control_v11e_sd15_shuffle"
controlnet_shuffle = ControlNetModel.from_pretrained(checkpoint_shuffle, torch_dtype=torch.float16)

checkpoint_pix = "lllyasviel/control_v11e_sd15_ip2p"
controlnet_pix = ControlNetModel.from_pretrained(checkpoint_pix, torch_dtype=torch.float16)


In [41]:
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base", torch_dtype=torch.float16)
pipe = pipe.to(device)



/home/matteo/anaconda3/lib/python3.9/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [42]:
pipe_img=StableDiffusionImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base", torch_dtype=torch.float16)
pipe_img = pipe_img.to(device)

In [43]:
checkpoint = "lllyasviel/control_v11f1p_sd15_depth"

controlnet = ControlNetModel.from_pretrained(checkpoint, torch_dtype=torch.float16)

depth_pipe  = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=[controlnet,controlnet_pix,controlnet_shuffle], torch_dtype=torch.float16
)

depth_pipe.control_net=[controlnet,controlnet_shuffle]

depth_pipe.to(device)

depth_pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


#### compute latents

In [44]:
gaussian_blur=torchvision.transforms.GaussianBlur(kernel_size=5)

In [45]:
z=get_latents(test_fmri_norm.numpy(),brain_to_latent,shapes,adjust=latent_adjust_values)


In [46]:

with torch.no_grad():

    guess_img=decode_with_partial_sampling(vae.decoder,{k:v.to(device).float() for k,v in z.items()},keep=len(z[0]))
    guess_img=upsample(guess_img).clamp(0,1)
    print(guess_img.max())
    guessed=[to_pil(i) for i in guess_img]

/home/matteo/anaconda3/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor(0.)


In [47]:
depth_z=get_latents(test_fmri_norm.numpy(),brain_to_depth,shapes,adjust=latent_depth_adjust_values)
with torch.no_grad():


    # guess_img=upsample(autoencoder.decoder.double()(z.to(device)).cpu())
    guess_img=decode_with_partial_sampling(vae.decoder,{k:v.to(device).float() for k,v in depth_z.items()},keep=len(z[0]))
    # img_out=pipe_embed.vae.float().decode(z.float().to(device)).sample.cpu()
    print(guess_img.max())
    guess_img=upsample(gaussian_blur(guess_img)).clamp(0,1)
    depth_guessed=[to_pil(i).convert("L").convert("RGB").filter(ImageFilter.SMOOTH_MORE) for i in guess_img]

NameError: name 'brain_to_depth' is not defined

In [ ]:
depth_images_quantized=[i.quantize(64).convert("RGB").filter(ImageFilter.SMOOTH_MORE) for i in depth_guessed]


#### Process the entire test set
* Base pipeline
* Depth
* Depth with styles
* Save caption from image
* Save caption from brain

In [ ]:
cols=["Caption from image","Caption from brain", "Reference image","Init Reconstruction","Depth Reconstruction",
      "SD_img_to_img","SD_only_text","SD_controldepth",f"SD_control",f"SD_control_{styles[0]}",
      f"SD_control_{styles[1]}",f"SD_control_{styles[2]}",f"SD_control_{styles[3]}",f"SD_control_{styles[4]}",
      f"SD_control_{styles[5]}"]
table = wandb.Table(columns=cols)


In [ ]:
for idx in range(len(test_dataset)):

    print(f"processing {idx+1}/{len(test_dataset)}")
    text_image = pipe(captions_from_brain[idx], guidance_scale=9, negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality"
    ).images
    img_image= pipe_img(captions_from_brain[idx],image=guessed[idx], strength=0.6, guidance_scale=9, negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality"
    ).images

    multi_cond_images=[depth_guessed[idx].quantize(64).convert("RGB").filter(ImageFilter.SMOOTH_MORE),guessed[idx],guessed[idx]]

    control_image=depth_pipe(captions_from_brain[idx], num_inference_steps=30,  image=multi_cond_images,
                           guidance_scale=9,negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality",
                          controlnet_conditioning_scale=[0.7,0.2,0.5]).images

    depth_image=depth_pipe(captions_from_brain[idx], num_inference_steps=30,  image=multi_cond_images,
                           guidance_scale=9,negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality",
                          controlnet_conditioning_scale=[0.7,0,0]).images

    style_image=[]
    for style in enumerate(styles):
        style_prompt=captions_from_brain[idx]+f",{style}."
        style_image+=depth_pipe(style_prompt, num_inference_steps=30,  image=multi_cond_images,
                           guidance_scale=9,negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality",
                          controlnet_conditioning_scale=[0.7,0,0]).images

    style_imgs=(wandb.Image(x) for x in style_image)
    table.add_data(captions_from_images[idx],captions_from_brain[idx],
                   wandb.Image(test_imgs[idx].permute(1,2,0).numpy()), #ref
                   wandb.Image(guessed[idx]), #init
                   wandb.Image(depth_guessed[idx]), #depth
                   wandb.Image(img_image[0]),
                   wandb.Image(text_image[0]),
                   wandb.Image(depth_image[0]),
                   wandb.Image(control_image[0]),
                   *style_imgs
                  )
    

In [ ]:
wandb.log({"table":table})

#### Example

In [ ]:
idx=np.random.randint(len(test_dataset))



print(captions_from_brain[idx])
print(captions_from_images[idx])
print(test_captions[idx])


text_image = pipe(captions_from_brain[idx], guidance_scale=9, negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality"
).images
img_image= pipe_img(captions_from_brain[idx],image=guessed[idx], strength=0.6, guidance_scale=9, negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality"
).images

multi_cond_images=[depth_guessed[idx].quantize(64).convert("RGB").filter(ImageFilter.SMOOTH_MORE),guessed[idx],guessed[idx]]

control_image=depth_pipe(captions_from_brain[idx], num_inference_steps=30,  image=multi_cond_images,
                       guidance_scale=9,negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality",
                      controlnet_conditioning_scale=[0.7,0.2,0.5]).images

depth_image=depth_pipe(captions_from_brain[idx], num_inference_steps=30,  image=multi_cond_images,
                       guidance_scale=9,negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality",
                      controlnet_conditioning_scale=[0.7,0,0]).images

style_image=[]
for style in enumerate(styles):
    style_prompt=captions_from_brain[idx]+f",{style} style."
    style_image+=depth_pipe(style_prompt, num_inference_steps=30,  image=multi_cond_images,
                       guidance_scale=9,negative_prompt = "monochrome, lowres, bad anatomy, worst quality, low quality",
                      controlnet_conditioning_scale=[0.7,0,0]).images


In [ ]:

# Create the figure with 3 columns and 10 rows
fig, ax_row = plt.subplots(nrows=3, ncols=5, figsize=(30,30))



ax_row[0,0].set_title("Caption from image")
ax_row[0,1].set_title("Reference image",color="red")
ax_row[0,2].set_title("Init Reconstruction")
ax_row[0,3].set_title("Depth")
ax_row[0,4].set_title("SD_img_to_img")


ax_row[1,0].set_title("Caption from brain")
ax_row[1,1].set_title("SD_only_text")
ax_row[1,2].set_title("SD_controldepth")
ax_row[1,3].set_title(f"SD_control")
ax_row[1,4].set_title(f"SD_control_{styles[0]}")


ax_row[2,0].set_title(f"SD_control_{styles[1]}")
ax_row[2,1].set_title(f"SD_control_{styles[2]}")
ax_row[2,2].set_title(f"SD_control_{styles[3]}")
ax_row[2,3].set_title(f"SD_control_{styles[4]}")
ax_row[2,4].set_title(f"SD_control_{styles[5]}")

# ax_row[2,1].set_title("SD_depth")
# ax_row[2,1].set_title("SD_depth")
# ax_row[2,2].set_title("SD_controlnet")
# ax_row[2,3].set_title(f"SD_controlnet_{styles[0]}")





ax_row[0,0].text(0.5, 0.5, captions_from_images[idx], ha='center', va='center', fontsize=15, wrap=True)
ax_row[0,0].axis('off')

ax_row[1,0].text(0.5, 0.5, captions_from_brain[idx], ha='center', va='center', fontsize=15, wrap=True)
ax_row[1,0].axis('off')

# wrapText(ax_row)

# Add the images to the second and third columns
ax_row[0,1].imshow(test_imgs[idx].permute(1,2,0))
ax_row[0,1].axis('off')

ax_row[0,2].imshow(guessed[idx])
ax_row[0,2].axis('off')

ax_row[0,3].imshow(depth_guessed[idx])
ax_row[0,3].axis('off')

ax_row[0,4].imshow(img_image[0])
ax_row[0,4].axis('off')

## second row

ax_row[1,1].imshow(text_image[0])
ax_row[1,1].axis('off')

ax_row[1,2].imshow(depth_image[0])
ax_row[1,2].axis('off')

ax_row[1,3].imshow(control_image[0])
ax_row[1,3].axis('off')

ax_row[1,3].imshow(style_image[0])
ax_row[1,3].axis('off')

ax_row[1,4].imshow(style_image[1])
ax_row[1,4].axis('off')

## third row

ax_row[2,0].imshow(style_image[1])
ax_row[2,0].axis('off')

ax_row[2,1].imshow(style_image[2])
ax_row[2,1].axis('off')

ax_row[2,2].imshow(style_image[3])
ax_row[2,2].axis('off')

ax_row[2,3].imshow(style_image[4])
ax_row[2,3].axis('off')

ax_row[2,4].imshow(style_image[5])
ax_row[2,4].axis('off')



# Adjust the spacing between subplots
fig.subplots_adjust(hspace=0.5, wspace=0.2)
# plt.savefig("captions.png")

In [ ]:
len(styles)